# Helpdesk #

I've had a request to de-randomize the desk schedule. Let me explain how I'm currently scheduling things and we can discuss where to go from here. For those curious, an example schedule for April and the code I used to generate it are at the bottom of the page.

## Current Goals ##

My current goals for the schedule are as follows:
1. No early shift people work afternoon desk shifts. No late shifters work mornings.
2. The morning and afternoon shifts won't be from the same county.
3. Everyone works the same number of shifts per month, or as close to that as possible.

## Current Strategy ##

1. Get a list of us in no particular order.
2. Add any days off you've requested.
3. Take into account imbalances in shifts worked. (IE, if Rick worked 1 more shift than anyone else last month, add that)
3. Loop through the days of the month and try the following:
  1. Take the first person in the list who can work that morning.
  2. Pair them with each subsequent person and see if that pair, day, and shift is ok with everyone.
4. Output either the pair I found or "None"

## The Random Problem ##

So, this results in a rather shuffled schedule. If we just had 7 workers and 2 shifts/day, we could have a schedule that loops every 7 work days. I don't know if there exists a schedule that loops taking all 3 goals into account. Additionally, there are some complications:
1. Time off. Whenever someone takes time off, it throws our cycle off. Not that this is typical, but this month had 10 days off to account for. Do we want to just "forgive" people those missed shifts to preserve our cycle? That is, do we want to throw out goal 3?
2. One of Neander's goals for this was to get us to interact with the other counties more often.

If you can find a more predictable schedule that fulfills our goals and makes Neander happy, let me know.

## Code ##

Here's the code I'm using to do this. Under it is sample output. Thanks to Steven, one of my Googler buddies, for helping me out with it. It's relatively straightforward if you've done any object-oriented programming. If you have questions, I'd be happy to answer them. If you want to fiddle with it yourself, I can give you the iPython/Jupyter notebook I cooked this up in.

In [7]:
weekends = {2, 3, 9, 10, 16, 17, 23, 24}  # April

class DeskJockey():
    def __init__(self, name, county, shift='8-5', days_off=weekends):
        self.name = name
        self.county = county
        self.shift = shift
        self.days_off = weekends.union({})  # dereference
        self.shifts_worked = 0

    def can_work(self, day, shift, partner=None):
        '''Checks if this jockey can work on a given day, in a given shift,
        (optionally) with a certain partner.'''
        if day in self.days_off:
            return False
        elif partner and self.county == partner.county:
            return False
        elif self.shift == '8-5' or self.shift == shift:
            return True
        else:
            # Don't know how you got here. :)
            return False
    
    def take_days_off(self, days):
        '''Takes a set of days to not be scheduled and stores it.'''
        self.days_off = self.days_off.union(days)
    
    def work(self):
        '''Just keeping track of number of shifts worked.
        This count isn't used in our decision logic. It's just record-keeping.'''
        self.shifts_worked += 1

    # The following two methods let us print() our objects in an intelligible form.
    def __str__(self):
        return(self.name)
    
    def __repr__(self):
        return("<" + self.__str__() + ">")


def schedule_day(jockeys, day):
    '''Pick a pair of workers for a day
    Given a list of jockeys and a day of the month, find a pair that can work on that day.
    
    Does so by looping over the list, one by one, and finding out if there are any eligible partners for that candidate.
    If we reach the end of the list, no one will work that day.
    
    This algorithm isn't totally optimal/fair. If a given afternoon worker has worked more shifts than anyone else,
    they might still be chosen due to being the first eligible match for a better morning worker.
    Still, it works pretty well and was easier to implement.
    
    Caution: This isn't a pure function. It modifies our workers.
    '''
    to_check = jockeys+[]  # dereference
    while to_check:
        am_candidate = to_check.pop(0)
        while not am_candidate.can_work(day, 'AM'):
            if to_check:
                am_candidate = to_check.pop(0)
            else:  # No one left to check
                return(None)
        for pm_candidate in jockeys:
            if pm_candidate.can_work(day, 'PM', am_candidate):
                am_candidate.work()
                pm_candidate.work()
                desk_jockeys.sort(key=lambda jockey: jockey.shifts_worked)
                # Let's ensure our candidates don't get picked tomorrow
                desk_jockeys.append(desk_jockeys.pop(desk_jockeys.index(am_candidate)))
                desk_jockeys.append(desk_jockeys.pop(desk_jockeys.index(pm_candidate)))
                return([am_candidate, pm_candidate])

    return(None)

desk_jockeys = [DeskJockey(name='Brandon',
                          county='Lake',
                          shift='AM'),
                DeskJockey('David',
                          'Lake',
                          'PM'),
                DeskJockey('Brad',
                          'Marion',
                          'AM'),
                DeskJockey('Kody',
                          'Marion',
                          'PM'),
                DeskJockey('John',
                          'Hernando'),
                DeskJockey('Rick',
                          'Hernando'),
                DeskJockey('Ronan',
                          'Citrus')]

from random import randint
for j in desk_jockeys:
    # Let's add some random days off, as a simulation.
    # 3 per person, and overlap with weekends may occur.
    j.take_days_off({randint(1,30), randint(1,30), randint(1,30)})

# And I'll say that Rick and Ronan worked one more shift than anyone else in March.
# They didn't, but this is just an example.
desk_jockeys[5].work()
desk_jockeys[6].work()
    
# Here to the end is just outputting the schedule as it's generated.
print('Start state:')
print('Name\tCounty\tShift\tDays off')
for j in desk_jockeys:
    print('%s\t%s\t%s\t%s' % (j.name, j.county[:6], j.shift, j.days_off))

print('\n\nApril Schedule:')
print('Day\tAM\tPM')
for day in range(1, 30+1):
    workpair = schedule_day(desk_jockeys, day)
    if workpair is None:
        print("NONE")
    else:
        print("%s\t%s\t%s" % (day, workpair[0], workpair[1]))

print('\nNumber of shifts worked:')
print(', '.join('%s: %s' % (jockey.name, jockey.shifts_worked) for jockey in desk_jockeys))

Start state:
Name	County	Shift	Days off
Brandon	Lake	AM	{2, 3, 7, 9, 10, 14, 16, 17, 23, 24}
David	Lake	PM	{2, 3, 4, 9, 10, 16, 17, 21, 23, 24}
Brad	Marion	AM	{2, 3, 7, 9, 10, 13, 16, 17, 23, 24}
Kody	Marion	PM	{2, 3, 9, 10, 12, 16, 17, 23, 24}
John	Hernan	8-5	{2, 3, 6, 9, 10, 16, 17, 23, 24, 30}
Rick	Hernan	8-5	{2, 3, 6, 9, 10, 16, 17, 21, 23, 24, 29}
Ronan	Citrus	8-5	{2, 3, 5, 9, 10, 14, 16, 17, 18, 23, 24}


April Schedule:
Day	AM	PM
1	Brandon	Kody
NONE
NONE
4	Brad	John
5	Rick	David
6	Ronan	Kody
7	John	David
8	Brandon	Rick
NONE
NONE
11	Brad	Ronan
12	John	David
13	Brandon	Kody
14	Brad	Rick
15	Ronan	John
NONE
NONE
18	Brandon	Kody
19	Brad	David
20	Rick	Ronan
21	John	Kody
22	Brandon	Rick
NONE
NONE
25	Brad	David
26	Ronan	John
27	Brandon	Kody
28	Brad	David
29	Ronan	John
30	Rick	Kody

Number of shifts worked:
Brandon: 6, Brad: 6, David: 6, Ronan: 7, John: 7, Rick: 7, Kody: 7
